이를 위해 pyspark과 Py4J 패키지를 설치한다. Py4J 패키지는 파이썬 프로그램이 자바가상머신상의 오브젝트들을 접근할 수 있게 해준다. Local Standalone Spark을 사용한다.

In [1]:
!pip install pyspark==3.3.1 py4j==0.10.9.5 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 9.6 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=ab316fa9de4d46b25066dfe942c5c8dce71a511b10beeec1507edf12ac6dab90
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


**Spark Session**을 하나 만든다

In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *

if __name__ == "__main__":
    spark = SparkSession \
        .builder \
        .appName("Spark Writing Demo") \
        .master("local[3]") \
        .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.3.1") \
        .getOrCreate()

In [2]:
!wget https://pyspark-test-sj.s3.us-west-2.amazonaws.com/appl_stock.csv

--2023-07-06 08:11:23--  https://pyspark-test-sj.s3.us-west-2.amazonaws.com/appl_stock.csv
Resolving pyspark-test-sj.s3.us-west-2.amazonaws.com (pyspark-test-sj.s3.us-west-2.amazonaws.com)... 52.92.163.234, 52.218.153.97, 52.92.129.106, ...
Connecting to pyspark-test-sj.s3.us-west-2.amazonaws.com (pyspark-test-sj.s3.us-west-2.amazonaws.com)|52.92.163.234|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 143130 (140K) [text/csv]
Saving to: ‘appl_stock.csv’

appl_stock.csv      100%[===================>] 139.78K   436KB/s    in 0.3s    

2023-07-06 08:11:24 (436 KB/s) - ‘appl_stock.csv’ saved [143130/143130]



In [3]:
df = spark.read \
    .format("csv") \
    .load("appl_stock.csv")

In [4]:
print("Num Partitions before: " + str(df.rdd.getNumPartitions()))
df.groupBy(spark_partition_id()).count().show()

Num Partitions before: 1
+--------------------+-----+
|SPARK_PARTITION_ID()|count|
+--------------------+-----+
|                   0| 1763|
+--------------------+-----+



In [5]:
df2 = df.repartition(4) # 파티션 수 재조정
print("Num Partitions after: " + str(df2.rdd.getNumPartitions()))
df2.groupBy(spark_partition_id()).count().show()

Num Partitions after: 4
+--------------------+-----+
|SPARK_PARTITION_ID()|count|
+--------------------+-----+
|                   0|  440|
|                   1|  441|
|                   2|  441|
|                   3|  441|
+--------------------+-----+



In [6]:
df3 = df2.coalesce(2) # 파티션 수를 줄이는 역활
print("Num Partitions after: " + str(df3.rdd.getNumPartitions()))
df3.groupBy(spark_partition_id()).count().show()

Num Partitions after: 2
+--------------------+-----+
|SPARK_PARTITION_ID()|count|
+--------------------+-----+
|                   0|  881|
|                   1|  882|
+--------------------+-----+



In [17]:
df.write \
    .format("avro") \
    .mode("overwrite") \
    .option("path", "dataOutput/avro/") \
    .save()

In [18]:
df2.write \
    .format("parquet") \
    .mode("overwrite") \
    .option("path", "dataOutput/parquet/") \
    .save()

In [19]:
df3.write \
    .format("json") \
    .mode("overwrite") \
    .option("path", "dataOutput/json/") \
    .save()

In [20]:
!ls -tl dataOutput/

total 0
drwxr-xr-x  8 jovyan users 256 Jul  6 08:17 json
drwxr-xr-x 12 jovyan users 384 Jul  6 08:17 parquet
drwxr-xr-x  6 jovyan users 192 Jul  6 08:17 avro


In [21]:
!ls -tl dataOutput/parquet/

total 112
-rw-r--r-- 1 jovyan users     0 Jul  6 08:17 _SUCCESS
-rw-r--r-- 1 jovyan users 28024 Jul  6 08:17 part-00003-c02ecb4a-f42c-4126-ab1d-1871cf30698b-c000.snappy.parquet
-rw-r--r-- 1 jovyan users 28069 Jul  6 08:17 part-00002-c02ecb4a-f42c-4126-ab1d-1871cf30698b-c000.snappy.parquet
-rw-r--r-- 1 jovyan users 27833 Jul  6 08:17 part-00000-c02ecb4a-f42c-4126-ab1d-1871cf30698b-c000.snappy.parquet
-rw-r--r-- 1 jovyan users 27787 Jul  6 08:17 part-00001-c02ecb4a-f42c-4126-ab1d-1871cf30698b-c000.snappy.parquet


In [22]:
!ls -tl dataOutput/avro/

total 88
-rw-r--r-- 1 jovyan users     0 Jul  6 08:17 _SUCCESS
-rw-r--r-- 1 jovyan users 89029 Jul  6 08:17 part-00000-02c960af-589b-4a06-8281-bae1884a68c8-c000.avro


In [23]:
!ls -tl dataOutput/json/

total 240
-rw-r--r-- 1 jovyan users      0 Jul  6 08:17 _SUCCESS
-rw-r--r-- 1 jovyan users 122570 Jul  6 08:17 part-00000-e9008506-50b0-4c8b-8ab8-4c0102b3a9c0-c000.json
-rw-r--r-- 1 jovyan users 122814 Jul  6 08:17 part-00001-e9008506-50b0-4c8b-8ab8-4c0102b3a9c0-c000.json
